## LangChain 구성요소 - 웹검색

In [1]:
import os, openai
import dotenv
from langchain.tools import Tool
from dotenv import load_dotenv
from langchain.chains.base import Chain
from langchain.agents import Tool
from googleapiclient.discovery import build


dotenv.load_dotenv('../.env')
GOOGLE_API_KEY = os.getenv("GOOGLE-SEARCH-API-KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH-ENGINE-ID")

openai.api_key = os.getenv("CHATGPT-RECEIPT")

In [28]:
def search_with_google_api(query, num_results=5):
    api_key = GOOGLE_API_KEY  # Google API 키
    cse_id = SEARCH_ENGINE_ID   # Custom Search Engine ID
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        results = service.cse().list(q=query, cx=cse_id, num=num_results).execute()
        print('results = ', results)
        items = results.get("items", [])
        for item in items:
            print(f"{item['title']}: {item['link']},  {item.get('snippet')}")
        return [
            {
                "title": item["title"], "snippet": item.get("snippet")  # 본문 내용 추가
            } 
            for item in items if item.get("snippet") is not None
        ]
    except Exception as e:
        return f'Google Search API Error: {e}'


class SearchChain(Chain):
    def __init__(self, tool):
        super().__init__()
        self._tool = tool

    @property
    def input_keys(self):
        return ["business_name"]

    @property
    def output_keys(self):
        return ["search_results"]

    def _call(self, inputs, **kwargs):
        business_name = inputs["business_name"]
        search_results = self._tool.func(business_name)
        return {"search_results": search_results}


# Tool로 검색 기능 정의
search_tool = Tool(
    name = "SearchBusinessCategory",
    func = lambda business_name: "\n".join(
        list([x['snippet'] for x in search_with_google_api(business_name)])
    ),
    description="상호명을 검색하여 관련 업종 정보를 제공합니다."
)

# Search chain
search_chain = SearchChain(search_tool)

In [29]:
# Search chain invoke 예제 코드

# 검색할 상호명
business_name = "에어부산"

# Search chain 호출
inputs = {"business_name": business_name}
result = search_chain.invoke(inputs)

# 결과 출력
print(result["search_results"])


results =  {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - 에어부산', 'totalResults': '10500000', 'searchTerms': '에어부산', 'count': 5, 'startIndex': 1, 'inputEncoding': 'utf8', 'outputEncoding': 'utf8', 'safe': 'off', 'cx': '50aee7f4193e043e4'}], 'nextPag